# Gathering data

In [ ]:
import zipfile
from io import BytesIO
from urllib.request import urlopen
import glob
import os

In [ ]:
def reading_file_data_from_zip_url(URL, FILENAME):
  data = BytesIO()
  with urlopen(URL) as zip_file_ref:
    with zipfile.ZipFile(BytesIO(zip_file_ref.read()), 'r') as zip_ref:
      data = zip_ref.read(FILENAME)
  return data

In [ ]:
ZIP_FILE_PATH = "https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip"
RAW_FILENAME = "household_power_consumption.txt"
rawdata = reading_file_data_from_zip_url(ZIP_FILE_PATH, RAW_FILENAME)

FILENAME = 'temp.csv'
with open(FILENAME, 'w') as fptr:
  fptr.writelines(rawdata.decode())

# Data Ingestion

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv(FILENAME, sep=';', na_values='?')

In [ ]:
# DATA CLEANUP

os.remove(FILENAME)

In [ ]:
df.shape

(2075259, 9)

In [ ]:
df = df.sample(frac=0.05)

# EDA

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103763 entries, 1454623 to 1978057
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Date                   103763 non-null  object 
 1   Time                   103763 non-null  object 
 2   Global_active_power    102500 non-null  float64
 3   Global_reactive_power  102500 non-null  float64
 4   Voltage                102500 non-null  float64
 5   Global_intensity       102500 non-null  float64
 6   Sub_metering_1         102500 non-null  float64
 7   Sub_metering_2         102500 non-null  float64
 8   Sub_metering_3         102500 non-null  float64
dtypes: float64(7), object(2)
memory usage: 7.9+ MB


In [ ]:
df.isna().sum()

Date                        0
Time                        0
Global_active_power      1263
Global_reactive_power    1263
Voltage                  1263
Global_intensity         1263
Sub_metering_1           1263
Sub_metering_2           1263
Sub_metering_3           1263
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

(102500, 9)

In [ ]:
df['Total_Sub_metering'] = df['Sub_metering_1'] + df['Sub_metering_2'] + df['Sub_metering_3']

In [ ]:
df.describe()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Total_Sub_metering
count,102500.000000,102500.000000,102500.000000,102500.000000,102500.000000,102500.000000,102500.000000,102500.000000
mean,1.085921,0.123673,240.843245,4.604425,1.091766,1.274302,6.422498,8.788566
std,1.050701,0.112544,3.246500,4.416879,6.086809,5.719327,8.419594,12.784714
min,0.078000,0.000000,224.240000,0.200000,0.000000,0.000000,0.000000,0.000000
25%,0.308000,0.048000,238.990000,1.400000,0.000000,0.000000,0.000000,0.000000
50%,0.596000,0.100000,241.010000,2.600000,0.000000,0.000000,1.000000,1.000000
75%,1.526000,0.194000,242.890000,6.400000,0.000000,1.000000,17.000000,18.000000
max,9.482000,1.130000,254.150000,41.200000,84.000000,76.000000,31.000000,127.000000


In [ ]:
df.corr()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Total_Sub_metering
Global_active_power,1.000000,0.249362,-0.398465,0.998870,0.483174,0.433916,0.641040,0.846323
Global_reactive_power,0.249362,1.000000,-0.113019,0.268565,0.130598,0.139466,0.087428,0.182146
Voltage,-0.398465,-0.113019,1.000000,-0.410149,-0.195222,-0.163648,-0.270423,-0.344246
Global_intensity,0.998870,0.268565,-0.410149,1.000000,0.488328,0.439572,0.628857,0.843283
Sub_metering_1,0.483174,0.130598,-0.195222,0.488328,1.000000,0.057042,0.105098,0.570833
Sub_metering_2,0.433916,0.139466,-0.163648,0.439572,0.057042,1.000000,0.083644,0.529600
Sub_metering_3,0.641040,0.087428,-0.270423,0.628857,0.105098,0.083644,1.000000,0.746023
Total_Sub_metering,0.846323,0.182146,-0.344246,0.843283,0.570833,0.529600,0.746023,1.000000


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df['Month'] = df['Date'].dt.month

In [ ]:
df['Year'] = df['Date'].dt.year

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102500 entries, 1454623 to 1978057
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Date                   102500 non-null  datetime64[ns]
 1   Time                   102500 non-null  object        
 2   Global_active_power    102500 non-null  float64       
 3   Global_reactive_power  102500 non-null  float64       
 4   Voltage                102500 non-null  float64       
 5   Global_intensity       102500 non-null  float64       
 6   Sub_metering_1         102500 non-null  float64       
 7   Sub_metering_2         102500 non-null  float64       
 8   Sub_metering_3         102500 non-null  float64       
 9   Total_Sub_metering     102500 non-null  float64       
 10  Year                   102500 non-null  int64         
 11  Month                  102500 non-null  int64         
dtypes: datetime64[ns](1), float64(8), int

In [ ]:
df.isna().sum()

Date                     0
Time                     0
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
Total_Sub_metering       0
Year                     0
Month                    0
dtype: int64

In [ ]:
df.duplicated().sum()

0

# Feature engineering

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train_test = df.sample(frac=0.2)

In [ ]:
df_train_test.columns

Index(['Date', 'Time', 'Global_active_power', 'Global_reactive_power',
       'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3', 'Total_Sub_metering', 'Year', 'Month'],
      dtype='object')

In [ ]:
independent_feature = ['Year', 'Month', 'Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity']
dependent_feature = ['Total_Sub_metering']

In [ ]:
X = df_train_test.loc[:, independent_feature]

In [ ]:
y = df_train_test.loc[:, dependent_feature]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

In [ ]:
print("X:", X_train.shape, X_test.shape)
print("y:", y_train.shape, y_test.shape)

X: (16400, 6) (4100, 6)
y: (16400, 1) (4100, 1)


# Bagging Regression

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor

In [ ]:
b_g = BaggingRegressor(base_estimator=SVR(),n_estimators=10, random_state=0)

In [ ]:
b_g.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:429: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


BaggingRegressor(base_estimator=SVR(), random_state=0)

In [ ]:
b_g.score(X_test, y_test)

-0.34368867942758996

In [ ]:
y_predict_b_g = b_g.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
print("MSE: %.2f" % mean_squared_error(y_test, y_predict_b_g))
print("MAE: %.2f" % mean_absolute_error(y_test, y_predict_b_g))

MSE: 232.42
MAE: 8.60


# Extra Trees Regression

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
e_t_r = ExtraTreesRegressor(n_estimators=10, random_state=0)

In [ ]:
e_t_r.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


ExtraTreesRegressor(n_estimators=10, random_state=0)

In [ ]:
e_t_r.score(X_test, y_test)

0.7660526543428826

In [ ]:
y_predict_e_t_r = e_t_r.predict(X_test)

In [ ]:
print("MSE: %.2f" % mean_squared_error(y_test, y_predict_e_t_r))
print("MAE: %.2f" % mean_absolute_error(y_test, y_predict_e_t_r))

MSE: 40.47
MAE: 3.10


# Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
r_f_r = RandomForestRegressor(n_estimators=10, random_state=0)

In [ ]:
r_f_r.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(n_estimators=10, random_state=0)

In [ ]:
r_f_r.score(X_test, y_test)

0.7717904033524223

In [ ]:
y_predict_r_f_r = r_f_r.predict(X_test)

In [ ]:
print("MSE: %.2f" % mean_squared_error(y_test, y_predict_r_f_r))
print("MAE: %.2f" % mean_absolute_error(y_test, y_predict_r_f_r))

MSE: 39.47
MAE: 3.08


# Stacking Regressor

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR

In [ ]:
estimators = [
    ('lr', RidgeCV()),
    ('svr', LinearSVR(random_state=42))
]

In [ ]:
s_r = StackingRegressor(
    estimators=estimators, 
    final_estimator=RandomForestRegressor(n_estimators=10, random_state=42)
)

In [ ]:
s_r.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:758: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklear

StackingRegressor(estimators=[('lr', RidgeCV(alphas=array([ 0.1,  1. , 10. ]))),
                              ('svr', LinearSVR(random_state=42))],
                  final_estimator=RandomForestRegressor(n_estimators=10,
                                                        random_state=42))

In [ ]:
s_r.score(X_test, y_test)

0.6970710895803988

In [ ]:
y_predict_s_r = s_r.predict(X_test)

In [ ]:
print("MSE: %.2f" % mean_squared_error(y_test, y_predict_s_r))
print("MAE: %.2f" % mean_absolute_error(y_test, y_predict_s_r))

MSE: 52.40
MAE: 4.06


# Voting Regression

In [ ]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import ElasticNet

In [ ]:
v_estimators = [
    ('er', ElasticNet()),
    ('lr', SVR()),
    ('rfr', RandomForestRegressor(n_estimators=10, random_state=1))
]

In [ ]:
v_r = VotingRegressor(v_estimators)

In [ ]:
v_r.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_voting.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


VotingRegressor(estimators=[('er', ElasticNet()), ('lr', SVR()),
                            ('rfr',
                             RandomForestRegressor(n_estimators=10,
                                                   random_state=1))])

In [ ]:
v_r.score(X_test, y_test)

0.6360585718881523

In [ ]:
y_predict_v_r = v_r.predict(X_test)

In [ ]:
print("MSE: %.2f" % mean_squared_error(y_test, y_predict_v_r))
print("MAE: %.2f" % mean_absolute_error(y_test, y_predict_v_r))

MSE: 62.95
MAE: 4.76


**==========THE END==========**